## import libraries

In [1]:
import numpy as np
import statsmodels.api as sm
import scipy as sp
import scipy.special
import time
import pandas as pd

## import dataset

In [2]:
data = sm.datasets.star98.load()
data.exog = sm.add_constant(data.exog, prepend = False)

exog = data.exog.to_numpy()
endog = data.endog.to_numpy()

# np.savetxt("star98_exog.csv", exog, delimiter=",")
# np.savetxt("star98_endog.csv", endog, delimiter=",")

AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

## define functions

In [ ]:
# sigmoid function
def sigmoid(x):
    return (1/(1 + np.exp(-x)))

# weight dot independent variables 
def pro(w, x):
    return sigmoid(np.inner(w, x))

# first coefficient in beta prior
def alpha(p, phi):
    return (((1 / phi) - 1) * p)

# second coefficient in beta prior
def beta(p, phi):
    return (((1 / phi) - 1) * (1 - p))

# average cost of beta-binomial regression excluding constant 
def cost(f, exog, endog):
    cos = 0
    for n in range(len(exog)):
        p = pro(f[:-1], exog[n])
        a = alpha(p, f[-1])
        b = beta(p, f[-1])
        cos += sp.special.betaln(endog[n][0] + a, endog[n][1] + b) - sp.special.betaln(a, b)
    return - cos / len(exog)

In [ ]:
# predict the output given the parametres
def predict(f, exog, endog):
    p_pred = sigmoid(np.matmul(exog, f[:-1]))
    return np.concatenate(([p_pred * (endog[:, 0] + endog[:, 1])], [(1 - p_pred) * (endog[:, 0] + endog[:, 1])]), axis = 0).T 

In [ ]:
# log-likelihood of the data
def LL(f, exog, endog):
    return - cost(f, exog, endog) * len(exog) + sum(sp.special.gammaln(endog[:,0] + endog[:,1] + 1) - sp.special.gammaln(endog[:,0] + 1) - sp.special.gammaln(endog[:,1] + 1))

In [ ]:
# gradient of cost of beta-binomial regression, averaged over the number of observations
def avgrad(f, exog, endog):
    grad = np.zeros(len(f))
    phi = f[-1]
    for n in range(len(exog)):
        p = pro(f[:-1], exog[n])
        a = alpha(p, f[-1])
        b = beta(p, f[-1])
        nt = endog[n][0]
        nf = endog[n][1]
        dCda_1 = sp.special.digamma(a) - sp.special.digamma(a + b)
        dCda_2 = sp.special.digamma(a + b + nt + nf) - sp.special.digamma(a + nt)
        dCdb_1 = sp.special.digamma(b) - sp.special.digamma(a + b)
        dCdb_2 = sp.special.digamma(a + b + nt + nf) - sp.special.digamma(b + nf)
        dadp = (1/phi) - 1
        dbdp = 1 - (1/phi)
        dadphi = - p / (phi ** 2)
        dbdphi = (p - 1) / (phi ** 2)
        dpdw = exog[n] * p * (1 - p)
        dw = ((dCda_1 + dCda_2) * dadp + (dCdb_1 + dCdb_2) * dbdp) * dpdw
        dphi = (dCda_1 + dCda_2) * dadphi + (dCdb_1 + dCdb_2) * dbdphi
        df = np.concatenate((dw, dphi), axis = None)
        grad += df
        
    return grad / len(exog)

In [ ]:
# find the best initial guess for phi, with other weights being the binomial regression result
def initial(params, exog, endog):
    guess = np.arange(0.99999, 10, 0.1)
    cos = [0]
    while np.argmin(cos) == 0:
        guess = guess / 10
        cos = []
        for m in range(len(guess)): 
            f = np.concatenate((params, [guess[m]]))
            cos.append(cost(f, exog, endog))
    f = np.concatenate((params, [guess[np.argmin(cos)]]))
    # print("initial guess = ", f)
    # print("cost of initial guess =", cos[np.argmin(cos)])
    return f

## binomial regression by statmodels

In [ ]:
glm_binom = sm.GLM(endog, exog, family = sm.families.Binomial())
res = glm_binom.fit()

## optimization by gradient descent

In [ ]:
time = 1700
learnrate = 0.00000000001 
f = initial(res.params, exog, endog)

for t in range(time):
    if (t + 1) % 100 == 0:
         print("iteration {time}, current cost = {cost}".format(time = t + 1, cost = cost(f, exog, endog)))
    f -= avgrad(f, exog, endog) * learnrate
print("result =", f)

## optimization by scipy optimizer

In [ ]:
timme = []
cosst = []
method = "SLSQP"
for n in range(5):
    start_time = time.time()
    bnds = []
    for m in range(len(res.params)):
        bnds.append((- np.inf, np.inf))
    bnds.append((0,1))
    bnds_t = tuple(bnds)
    f = initial(res.params, exog, endog)
    ress = sp.optimize.minimize(cost, f, args = (exog, endog), method = method, bounds = bnds_t)
    # print("result =", ress.x)
    # print("cost =", cost(ress.x, exog, endog))
    # print("--- %s seconds ---" % (time.time() - start_time))
    timme.append(time.time() - start_time)
    cosst.append(cost(ress.x, exog, endog))
print("### average running time = {time:.2f}, average cost = {cost:.4f},".format(time = np.average(timme), cost = np.average(cosst)), "method =", method)

#### cost of initial guess = 541.2959


#### average running time = 10.45, average cost = 541.2591, method = Nelder-Mead
#### average running time = 1.20, average cost = 541.2739, method = Powell
#### average running time = 0.63, average cost = 541.2959, method = CG
#### average running time = 0.66, average cost = 541.2959, method = BFGS
#### average running time = 0.63, average cost = 541.2959, method = L-BFGS-B
#### average running time = 4.27, average cost = 541.2861, method = TNC
#### average running time = 0.47, average cost = 541.2959, method = COBYLA
#### average running time = 2.07, average cost = 541.2494, method = SLSQP
#### average running time = 8.61, average cost = 541.2959, method = trust-constr


## compare to aod package

In [ ]:
# aod
aod = pd.read_csv('param_aod.csv')
f = aod.iloc[:,1].to_numpy()
print("cost =", cost(f, exog, endog))
print("log-likelihood =", LL(f, exog, endog))

In [ ]:
# scipy optimizer
f = ress.x
print("cost =", cost(f, exog, endog))
print("log-likelihood =", LL(f, exog, endog))